<a href="https://colab.research.google.com/github/smitaforward/Data_Science/blob/master/Tensorflow_2_0_Structure_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



In [4]:
!pip install tensorflow==2.0.0-alpha0

    100% |████████████████████████████████| 79.9MB 408kB/s 
    100% |████████████████████████████████| 61kB 20.5MB/s 
    100% |████████████████████████████████| 3.0MB 11.1MB/s 
    100% |████████████████████████████████| 419kB 11.4MB/s 
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


In [0]:
import tensorflow as tf
from tensorflow import feature_column
from tensorflow import keras

# Prep Data

In [33]:
# read data in
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [0]:
# split the data into train, validate, test

train, test = train_test_split(dataframe, test_size = 0.2)

In [0]:
train, val = train_test_split(train, test_size = 0.2)

In [11]:
print(f'train example: {len(train)}')
print(f'validation example: {len(val)}')
print(f'test example: {len(test)}')

train example: 193
validation example: 49
test example: 61


In [19]:
df = dataframe.head(3)
df.pop('target')


0    0
1    1
2    0
Name: target, dtype: int64

In [0]:
# create an input pipline usng tf.data
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds 
  

In [0]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False,batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [22]:
train_ds

<BatchDataset shapes: ({age: (None,), sex: (None,), cp: (None,), trestbps: (None,), chol: (None,), fbs: (None,), restecg: (None,), thalach: (None,), exang: (None,), oldpeak: (None,), slope: (None,), ca: (None,), thal: (None,)}, (None,)), types: ({age: tf.int32, sex: tf.int32, cp: tf.int32, trestbps: tf.int32, chol: tf.int32, fbs: tf.int32, restecg: tf.int32, thalach: tf.int32, exang: tf.int32, oldpeak: tf.float32, slope: tf.int32, ca: tf.int32, thal: tf.string}, tf.int32)>

In [25]:

for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of cp:', feature_batch['cp'])
  print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([48 50 63 44 66], shape=(5,), dtype=int32)
A batch of cp: tf.Tensor([4 3 4 3 4], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([1 0 1 0 1], shape=(5,), dtype=int32)


In [0]:

# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [28]:
example_batch

{'age': <tf.Tensor: id=134, shape=(5,), dtype=int32, numpy=array([48, 50, 63, 44, 66], dtype=int32)>,
 'ca': <tf.Tensor: id=135, shape=(5,), dtype=int32, numpy=array([0, 0, 3, 0, 1], dtype=int32)>,
 'chol': <tf.Tensor: id=136, shape=(5,), dtype=int32, numpy=array([274, 219, 407, 141, 212], dtype=int32)>,
 'cp': <tf.Tensor: id=137, shape=(5,), dtype=int32, numpy=array([4, 3, 4, 3, 4], dtype=int32)>,
 'exang': <tf.Tensor: id=138, shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 1], dtype=int32)>,
 'fbs': <tf.Tensor: id=139, shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 0], dtype=int32)>,
 'oldpeak': <tf.Tensor: id=140, shape=(5,), dtype=float32, numpy=array([0.5, 1.6, 4. , 0.6, 0.1], dtype=float32)>,
 'restecg': <tf.Tensor: id=141, shape=(5,), dtype=int32, numpy=array([2, 0, 2, 0, 2], dtype=int32)>,
 'sex': <tf.Tensor: id=142, shape=(5,), dtype=int32, numpy=array([1, 0, 0, 0, 1], dtype=int32)>,
 'slope': <tf.Tensor: id=143, shape=(5,), dtype=int32, numpy=array([2, 2, 2, 2, 1], dtype

In [0]:
# choose columns to use

feature_columns = []
# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak','slope', 'ca']:
  feature_columns.append(feature_column.numeric_column(header))

#bucketized cols
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

#indicator cols
thal = feature_column.categorical_column_with_vocabulary_list('thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

#embedding cols

thal_embedding = feature_column.embedding_column(thal, dimension = 8)
feature_columns.append(thal_embedding)

#crossed cols
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)


In [54]:
feature_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='trestbps', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='chol', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='thalach', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='oldpeak', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='slope', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ca', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 BucketizedColumn(source_column=NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(18, 25, 30, 35, 40, 45, 50, 55, 60, 65)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='thal', vocabulary_list=('fixed', 'normal', 'reversi

# Model

In [0]:
#create a feature layer
feature_layer = keras.layers.DenseFeatures(feature_columns)

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [57]:
model=keras.Sequential([
    feature_layer,
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer = 'adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5
)

Epoch 1/5
7/7 [==============================] - 1s 96ms/step - loss: 1.2231 - accuracy: 0.6296 - val_loss: 0.6339 - val_accuracy: 0.7347
Epoch 2/5
7/7 [==============================] - 0s 42ms/step - loss: 0.4638 - accuracy: 0.7386 - val_loss: 0.5107 - val_accuracy: 0.7755
Epoch 3/5
7/7 [==============================] - 0s 44ms/step - loss: 0.4218 - accuracy: 0.7491 - val_loss: 0.5342 - val_accuracy: 0.7755
Epoch 4/5
7/7 [==============================] - 0s 38ms/step - loss: 0.4706 - accuracy: 0.7113 - val_loss: 0.9083 - val_accuracy: 0.5306
Epoch 5/5
7/7 [==============================] - 0s 35ms/step - loss: 0.6329 - accuracy: 0.6158 - val_loss: 0.6192 - val_accuracy: 0.6939


In [58]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 21ms/step - loss: 0.8185 - accuracy: 0.5902
Accuracy 0.59016395


In [0]:
def demo(feature_column):
  feature_layer = keras.layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [32]:

age = feature_column.numeric_column("age")
demo(age)

W0326 19:35:55.851586 140473493784448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:2758: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


[[48.]
 [50.]
 [63.]
 [44.]
 [66.]]


In [34]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

W0326 19:37:19.641683 140473493784448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:2902: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [35]:
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

W0326 19:38:20.607078 140473493784448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4307: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0326 19:38:20.608749 140473493784448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4362: VocabularyListCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [36]:

# Notice the input to the embedding column is the categorical column
# we previously created
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[ 0.16490324 -0.37901995 -0.43246165 -0.38170975 -0.23772152 -0.33532616
   0.21067953 -0.42689922]
 [-0.05292119  0.70207345  0.40566146  0.06471767 -0.1609036  -0.18644665
   0.49522772  0.16701943]
 [ 0.16490324 -0.37901995 -0.43246165 -0.38170975 -0.23772152 -0.33532616
   0.21067953 -0.42689922]
 [-0.05292119  0.70207345  0.40566146  0.06471767 -0.1609036  -0.18644665
   0.49522772  0.16701943]
 [-0.05292119  0.70207345  0.40566146  0.06471767 -0.1609036  -0.18644665
   0.49522772  0.16701943]]


In [37]:
thal_hashed = feature_column.categorical_column_with_hash_bucket(
      'thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

W0326 19:45:36.422112 140473493784448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4362: HashedCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [38]:
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))

W0326 19:46:50.836545 140473493784448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4362: CrossedColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
